In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
product_data = pd.read_csv("drive/My Drive/productsclassified.csv")

In [ ]:
candidate_labels = product_data.Classification.unique()
candidate_labels

array(['Boots', 'Jackets', 'Shoes', 'Fishing Tools', 'Pillows',
       'Suspenders', 'Towels', 'Shirts', 'Underwear', 'Tables', 'Socks',
       'Vests', 'Headwear', 'Slippers', 'Bicycles', 'Couches',
       'Snowshoes', 'Protective Clothing', 'Footwear Accessories',
       'Baselayers', 'Bags', 'Dog Accessories', 'Patches', 'Shorts',
       'Stoves', 'Locks', 'Pants', 'Games', 'Sunglasses', 'Lamps',
       'Portable Lights', 'Chairs', 'Swimwear', 'Wacthes', 'Scarves',
       'Sheets', 'Cabinets', 'Gloves', 'Rugs', 'Kayak Accessories',
       'Knives', 'Knife Accessories', 'Bedding Covers', 'Car Accessories',
       'Paddleboards', 'Dresses', 'Sleeping Bags', 'Binoculars', 'Kayaks',
       'Blankets', 'Bicycle Accessories', 'Shower Shelters', 'Belts'],
      dtype=object)

In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline
from tqdm.notebook import tqdm

In [ ]:
classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

In [ ]:
product_data.description

0      ['These waterproof hiking boots for men are ru...
1      ["The next level of weather protection. This l...
2      ['Great grip and extra breathability make thes...
3      ["This lightweight fly rod delivers outstandin...
4      ['Add a pop of paddling fun to your bed, chair...
                             ...                        
242    ['A hypoallergenic option with all the warmth,...
243    ['Crafted from premium denim that will last we...
244    ['Perfect to pair with our Vista Trekking Pant...
245    ['A new take on the classic rugby. Rugged as a...
246     ['Performance-inspired and made for every day.']
Name: description, Length: 247, dtype: object

In [ ]:
classifier_pipeline(product_data.description, candidate_labels)

# Point 2

In [ ]:
not_labeled = pd.read_csv("drive/My Drive/productsfull.csv")

In [ ]:
not_labeled

In [ ]:
list_of_colors = list(pd.read_html("https://simple.wikipedia.org/wiki/List_of_colors")[0].Name)

In [ ]:
for i in range(len(list_of_colors)):
   list_of_colors[i] = list_of_colors[i].lower()

In [ ]:
not_labeled["color_col"] = not_labeled.description.apply(lambda x: ' '.join([word for word in x.split() if word  in (list_of_colors)]))
len(not_labeled["color_col"])

25123

In [ ]:
not_labeled["color_col"].unique()

array(['', 'purple', 'red', 'blue', 'copper', 'black yellow', 'black',
       'orange', 'orange orange', 'coffee coffee coffee', 'coffee',
       'indigo blue', 'indigo', 'white', 'white white', 'salmon', 'gray',
       'rose'], dtype=object)

In [ ]:
df2=not_labeled.mask(not_labeled == "[]")
df2.isna().sum().sum()


62362

In [ ]:
df2 = df2.dropna()


In [ ]:
df2=df2.mask(not_labeled == "")
df2.isna().sum().sum()

516

In [ ]:
df2 = df2.dropna()


In [ ]:
df2 = df2.reset_index()
df2['color_col'][0]

'indigo blue'

In [ ]:
comparison_column = np.where((df2['color_col'] == df2['colorname']))
df2.comparison= comparison_column


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [ ]:
df2['coompared'] = df2.apply(lambda x: x['color_col'] if x['color_col'] ==
                     x['colorname'] else np.nan, axis=1)

In [ ]:
df2['coompared'].unique()

array([nan])

# Point 3

In [ ]:
df2.description[0]

"['True indigo dyes bring a new dimension to the cotton madras of this lightweight popover. Wonderfully airy and cool, and refreshingly new in rich blue hues and a slim pullover style.']"

In [ ]:
df2.description = df2.description.apply(lambda x: x.strip("['"))

In [ ]:
df2.description = df2.description.apply(lambda x: x.strip("']"))

In [ ]:
df2.description[0]


'True indigo dyes bring a new dimension to the cotton madras of this lightweight popover. Wonderfully airy and cool, and refreshingly new in rich blue hues and a slim pullover style.'

In [ ]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
def get_hotwords(text):
  result = []
  pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
  doc = nlp(text.lower()) 
  for token in doc:
    if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
      continue
    if(token.pos_ in pos_tag):
      result.append(token.text)
  return result

In [ ]:
df2['keywords'] = df2.description.apply(get_hotwords)

# Point 4

In [ ]:
# Train test validation Split
from sklearn.model_selection import train_test_split

In [ ]:
df = product_data[["description", "Classification"]]
df=df.mask(df == "[]")
print(df.isna().sum().sum())


1


In [ ]:
df = df.dropna()
df= df.mask(df == "")
df.isna().sum().sum()


0

In [ ]:
df = df.reset_index()
df

,index,description,Classification
0,0,['These waterproof hiking boots for men are ru...,Boots
1,1,"[""The next level of weather protection. This l...",Jackets
2,2,['Great grip and extra breathability make thes...,Shoes
3,3,"[""This lightweight fly rod delivers outstandin...",Fishing Tools
4,4,"['Add a pop of paddling fun to your bed, chair...",Pillows
...,...,...,...
241,242,"['A hypoallergenic option with all the warmth,...",Blankets
242,243,['Crafted from premium denim that will last we...,Pants
243,244,['Perfect to pair with our Vista Trekking Pant...,Belts
244,245,['A new take on the classic rugby. Rugged as a...,Shirts


In [ ]:
df.description = df.description.apply(lambda x: x.strip("['"))

In [ ]:
df.description = df.description.apply(lambda x: x.strip("']"))

In [ ]:
df.description = df.description.apply(lambda x: x.strip('"'))

In [ ]:
df.description [1]

"The next level of weather protection. This light-up jacket resists the elements and keeps you visible in low-light conditions. From running, biking or walking the dog, the durable construction and innovative safety features won't let you down."

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
data = df.description
labels = df.Classification

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( data, labels, test_size=0.20, random_state=42)

In [ ]:
candidate_labels = list(y_train.unique()) 
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = x_train.iloc[i,]
    cat = y_train.iloc[i,]
    res = classifier(text, candidate_labels, multi_class=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = []
    for i, (k,v) in enumerate(sorted_dict.items()):
        if(i > 2): #storing only the best 3 predictions
            break
        else:
            categories.append(k)
    predictedCategories.append(categories)
    trueCategories = y_train

  0%|          | 0/100 [00:00<?, ?it/s]

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf

In [ ]:
category_idx = {cat : i for i,cat in enumerate(y_train.unique())} 

In [ ]:
y_trueEncoded = []
y_predEncoded = []
for y_true, y_pred in zip(trueCategories, predictedCategories):
    encTrue = [0] * len(y_train)
    for cat in y_true:
        idx = category_idx[cat]
        encTrue[idx] = 1
    y_trueEncoded.append(encTrue)
    encPred = [0] * len(y_train)
    for cat in y_pred:
        idx = category_idx[cat]
        encPred[idx] = 1
    y_predEncoded.append(encPred)

KeyError: ignored